In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Read data

In [39]:
import pandas as pd

df= pd.read_csv("/content/drive/MyDrive/ds/data.csv",thousands=',',skipinitialspace=True)
df.head()

,title,description,price,square,estate_type,post_date,post_id,link,province,district,ward,Tầng,Số phòng ngủ
0,Chính chủ cần bán nhà 2 mặt ngõ phố Hoàn Cầu P...,Tôi cần bán nhà 2 mặt ngõ phố Hoàn Cầu Phường ...,6800000000.0,45.5,Mua bán nhà riêng,2023-10-20,580302,https://bds.com.vn/chinh-chu-can-ban-nha-2-mat...,Hà Nội,Đống Đa,Ô Chợ Dừa,4.0,NaN
1,CHUẨN SỐNG SANG - KHÔNG GIAN AN LÀNH chỉ có tạ...,CHÍNH SÁCH BÁN HÀNG HẤP DẪN NHẤT CHỈ CÓ TẠI TÒ...,3000000000.0,108.0,Mua bán căn hộ chung cư,2023-10-01,97579,https://bds.com.vn/chuan-song-sang---khong-gia...,Hà Nội,Hai Bà Trưng,Vĩnh Tuy,NaN,3.0
2,"Bán nhà đẹp Bồ Đề 36 m2 x 3 tầng x 2,6 tỉ , ng...","Bán nhà đẹp Bồ Đề 36 m2 x 3 tầng x 2,6 tỉ , ng...",2600000000.0,36.0,Mua bán nhà riêng,2023-10-01,87277,https://bds.com.vn/ban-nha-dep-bo-de-36m2-x-3-...,Hà Nội,Long Biên,Bồ Đề,3.0,3.0
3,"Nhà Bồ Đề siêu đẹp 32 m2 x 5 tầng x 3,2 tỉ , ô...","Bán nhà Bồ Đề siêu đẹp 32 m2 x 5 tầng x 3,2 tỉ...",3200000000.0,32.0,Mua bán nhà riêng,2023-10-01,87627,https://bds.com.vn/nha-bo-de-sieu-dep-32m2-x-5...,Hà Nội,Long Biên,Bồ Đề,5.0,3.0
4,Nhà Phan Đình Phùng – 2 thoáng – Phân Lô – ô t...,Mô tả : Do chuyển sang nước ngoài định cư cùng...,9000000000.0,42.0,Mua bán nhà riêng,2023-10-01,93144,https://bds.com.vn/nha-phan-dinh-phung-%E2%80%...,Hà Nội,Hoàn Kiếm,Hàng Mã,NaN,NaN


## Chuyển định dạng time về float

In [40]:
df['post_date'] = pd.to_datetime(df['post_date'])  # Chuyển cột 'post_date' thành kiểu datetime

# Chuyển datetime thành timestamp (float)
df['post_date_float'] = df['post_date'].astype(int) / 10**9
df['price'] = pd.to_numeric(df['price'], errors='coerce')

## Lấy ra các trường cần cho việc dự đoán

In [41]:
columns_to_keep = ['price', 'square', 'post_date_float', 'Tầng', 'Số phòng ngủ']

# Tạo DataFrame mới chỉ với các cột bạn quan tâm
df = df[columns_to_keep].copy()

In [42]:
df

,price,square,post_date_float,Tầng,Số phòng ngủ
0,6.800000e+09,45.5,1.697760e+09,4.0,NaN
1,3.000000e+09,108.0,1.696118e+09,NaN,3.0
2,2.600000e+09,36.0,1.696118e+09,3.0,3.0
3,3.200000e+09,32.0,1.696118e+09,5.0,3.0
4,9.000000e+09,42.0,1.696118e+09,NaN,NaN
...,...,...,...,...,...
63019,8.500000e+09,50.0,1.583539e+09,5.0,4.0
63020,3.450000e+09,35.0,1.583626e+09,5.0,3.0
63021,6.300000e+09,70.0,1.583712e+09,NaN,6.0
63022,9.000000e+09,186.0,1.583712e+09,36.0,4.0


## Tìm và xóa những trường bản ghi có giá trị NaN




In [ ]:
## Tìm những trường dữ liệu có giá trị NaN
nan_values = df.isna()
nan_columns = nan_values.any()
columns_with_nan = df.columns[nan_columns].tolist()
columns_with_nan

['price', 'square', 'Tầng', 'Số phòng ngủ']

In [ ]:
df.isna().sum()/df.shape[0]*100

price               4.146040
square              4.522087
post_date_float     0.000000
Tầng               24.658860
Số phòng ngủ       25.812389
dtype: float64

In [ ]:
per = 0.5
df_dropped = df.dropna(axis=1,thresh=int(df.shape[0]*per))
df_dropped_2 = df_dropped.dropna(how='any')
#df_dropped_2 = df_dropped.dropna(axis=0,thresh=int(df.shape[1]*0.5))
df_dropped_2

,price,square,post_date_float,Tầng,Số phòng ngủ
2,2.600000e+09,36.0,1.696118e+09,3.0,3.0
3,3.200000e+09,32.0,1.696118e+09,5.0,3.0
5,2.200000e+09,40.0,1.696118e+09,4.0,4.0
13,7.800000e+09,80.0,1.698970e+09,4.0,4.0
18,4.900000e+09,55.0,1.696118e+09,5.0,4.0
...,...,...,...,...,...
63018,1.750000e+09,39.0,1.583539e+09,3.0,3.0
63019,8.500000e+09,50.0,1.583539e+09,5.0,4.0
63020,3.450000e+09,35.0,1.583626e+09,5.0,3.0
63022,9.000000e+09,186.0,1.583712e+09,36.0,4.0


Tạo df_x là những trường đầu vào

In [ ]:
df_x = df_dropped_2.iloc[:, 1:9]
df_x

,square,post_date_float,Tầng,Số phòng ngủ
2,36.0,1.696118e+09,3.0,3.0
3,32.0,1.696118e+09,5.0,3.0
5,40.0,1.696118e+09,4.0,4.0
13,80.0,1.698970e+09,4.0,4.0
18,55.0,1.696118e+09,5.0,4.0
...,...,...,...,...
63018,39.0,1.583539e+09,3.0,3.0
63019,50.0,1.583539e+09,5.0,4.0
63020,35.0,1.583626e+09,5.0,3.0
63022,186.0,1.583712e+09,36.0,4.0


## Đầu ra là trường price

In [ ]:
df_y = df_dropped_2.iloc[:, 0]
df_y

2        2.600000e+09
3        3.200000e+09
5        2.200000e+09
13       7.800000e+09
18       4.900000e+09
             ...     
63018    1.750000e+09
63019    8.500000e+09
63020    3.450000e+09
63022    9.000000e+09
63023    2.480000e+10
Name: price, Length: 43456, dtype: float64

In [ ]:
data_dummies = pd.get_dummies(df_x, drop_first=True)
data_dummies = data_dummies.astype(float)
cols = data_dummies.columns.values
data_preprocessed = data_dummies[cols]
data_preprocessed

,square,post_date_float,Tầng,Số phòng ngủ
2,36.0,1.696118e+09,3.0,3.0
3,32.0,1.696118e+09,5.0,3.0
5,40.0,1.696118e+09,4.0,4.0
13,80.0,1.698970e+09,4.0,4.0
18,55.0,1.696118e+09,5.0,4.0
...,...,...,...,...
63018,39.0,1.583539e+09,3.0,3.0
63019,50.0,1.583539e+09,5.0,4.0
63020,35.0,1.583626e+09,5.0,3.0
63022,186.0,1.583712e+09,36.0,4.0


## Chuyển về dạng ma trận phục vụ cho việc dự đoán

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
scaler = StandardScaler()
scaler.fit(data_preprocessed)

StandardScaler()

In [ ]:
data_preprocessed = scaler.transform(data_preprocessed)
data_preprocessed

array([[-6.82502625e-03,  1.68239897e+00, -6.37735297e-01,
        -3.93539866e-01],
       [-6.82563944e-03,  1.68239897e+00, -8.89943971e-03,
        -3.93539866e-01],
       [-6.82441306e-03,  1.68239897e+00, -3.23317368e-01,
        -1.60645469e-01],
       ...,
       [-6.82517955e-03, -3.24995868e+00, -8.89943971e-03,
        -3.93539866e-01],
       [-6.80203150e-03, -3.24617039e+00,  9.73805635e+00,
        -1.60645469e-01],
       [-6.81674814e-03, -3.24617039e+00,  3.05518489e-01,
         1.00382651e+00]])

In [19]:
X,X_test,Y,Y_test = train_test_split(data_preprocessed,df_y,test_size=0.2)

## Ridge, Lasso, ElasticNet

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge

In [21]:
reg_ridge = Ridge(alpha = 1.0)
reg_ridge.fit(df_x, df_y)

# Sai số huấn luyện của mô hình trên tập train
print(reg_ridge.score(df_x, df_y))
# Hệ số hồi qui và hệ số chặn
print(reg_ridge.coef_)
print(reg_ridge.intercept_)

0.00014285098204613877
[-3.09769265e+06  1.66257558e+08 -3.77582473e+13 -2.00021100e+14]
-2.715987853597912e+17


In [22]:
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression
X, y = make_regression(noise=4, random_state=0)
reg_lasso_cv = LassoCV(cv=5, random_state=0).fit(X, y)
print(reg_lasso_cv.coef_)
print(reg_lasso_cv.intercept_)

[-4.21242132e-01 -0.00000000e+00  8.74020196e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  5.04074761e-02  7.46065852e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -1.72366886e-01  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -4.29663159e-01
  1.43615035e-01  0.00000000e+00 -1.79948525e-01  0.00000000e+00
 -0.00000000e+00  7.30847374e+01 -3.43884703e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.13286030e-01 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  3.94405369e+01 -0.00000000e+00 -0.00000000e+00
  5.23718682e+01  8.32674366e-01  4.35584487e+01 -0.00000000e+00
  0.00000000e+00  1.55124290e-01  2.58648431e-01 -0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  3.22013861e+01  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  1.55887

In [23]:
from sklearn.linear_model import Lasso

reg_lasso = Lasso(alpha = 1.0)
reg_lasso.fit(df_x, df_y)

# Sai số huấn luyện trên tập train
print(reg_lasso.score(df_x, df_y))

# Hệ số hồi qui và hệ số chặn
print(reg_lasso.coef_)
print(reg_lasso.intercept_)

0.00014285098204602775
[-3.09769287e+06  1.66257561e+08 -3.77582680e+13 -2.00021347e+14]
-2.7159878864761222e+17


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.245e+39, tolerance: 4.491e+35
  model = cd_fast.enet_coordinate_descent(


In [24]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

regr = ElasticNet(alpha = 1.0, l1_ratio=0.5, random_state=0)
regr.fit(df_x, df_y)

print(regr.score(df_x, df_y))
print(regr.score(X_test, Y_test))
print(regr.coef_)
print(regr.intercept_)

0.00014284596753955103
-0.2896000629303974
[-3.09296865e+06  1.66199312e+08 -3.72948150e+13 -1.94789467e+14]
-2.7152909407648278e+17


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.245e+39, tolerance: 4.491e+35
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error


models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('ElasticNet', ElasticNet()))

In [28]:
X,X_test,Y,Y_test = train_test_split(data_preprocessed,df_y,test_size=0.2,random_state=365)

In [31]:
rmses = []
mse = []
maes = []
names = []
for name, model in models:
    model.fit(X, Y)
    Y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(Y_test, Y_pred))
    rmses.append(rmse)

    mae = mean_absolute_error(Y_test, Y_pred)
    maes.append(mae)

    names.append(name)
    msg = "%s: RMSE:%f, MAE:%f" % (name, rmse, mae)
    print(msg)

LR: RMSE:85009185749294704.000000, MAE:5940373785844734.000000
Ridge: RMSE:85009179285484768.000000, MAE:5940288650284338.000000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.020e+39, tolerance: 4.428e+35
  model = cd_fast.enet_coordinate_descent(


Lasso: RMSE:85009185749294704.000000, MAE:5940373785844736.000000
ElasticNet: RMSE:84950226525753248.000000, MAE:5053350531913794.000000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.685e+39, tolerance: 4.428e+35
  model = cd_fast.enet_coordinate_descent(


## Random forest

In [32]:
X = df_x
y = df_y

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [34]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 42)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [35]:
y_pred = regressor.predict(X_test)

In [36]:
rf=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
rf

,Actual,Predicted
23201,9.700000e+09,9.548038e+09
9085,7.200000e+09,8.315428e+09
34348,3.200000e+09,8.458262e+09
49031,4.000000e+09,3.272173e+09
2722,5.200000e+09,5.729320e+09
...,...,...
50969,6.900000e+09,5.136367e+09
50124,5.900000e+08,7.277500e+09
38610,3.800000e+09,4.244402e+09
53324,3.400000e+09,3.719079e+09


In [37]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1.1774532568196952e+16
Mean Squared Error: 3.323599125365126e+35
Root Mean Squared Error: 5.765066457002144e+17
